In [ ]:
!pip install pyroomacoustics


In [1]:
import numpy as np
import pandas as pd
import librosa as lr
import glob
import os
from pyroomacoustics.experimental.rt60 import measure_rt60

from IPython.display import Audio

In [2]:
rir_df = pd.read_csv('/home/cxw/sonos_rirs/reverb_audio/080322_10K/audio_df.csv')

In [3]:
# sr = 16000
# rir_sample, _ = lr.load(rir_df.iloc[200]['file'], sr=sr)

In [4]:
def compute_rt60(rir_path, sr=16000):
    rir, _ = lr.load(rir_path, sr=sr)
    rt60 = measure_rt60(rir, fs=sr)
    return rt60

In [5]:
files = rir_df['file_rir'].tolist()
rt60_dict = {}
bad_files = []
for file in files:
    try:
        rt60 = compute_rt60(file)
    except:
        print(file)
        bad_files.append(file)
        continue
    rt60_dict[file] = rt60

In [6]:
rt60_list = []
for row in rir_df.iterrows():
    file = row[1]['file_rir']
    try:
        rt60 = compute_rt60(file)
    except:
        print(file)
        rt60 = None
        pass
    rt60_list.append(rt60)
rir_df['computed_rt60'] = rt60_list

In [14]:
rooms = rir_df['room'].unique()
gb_room = rir_df.groupby('room')
mean_rt60_dict = {}
for room in rooms:
    mean_rt60 = gb_room.get_group(room)['computed_rt60'].mean()
    mean_rt60_dict[room] = mean_rt60

In [15]:
rir_df['mean_rt60'] = rir_df['room'].apply(lambda row:mean_rt60_dict[row])
rir_df.to_csv('/home/cxw/sonos_rirs/reverb_audio/080322_10K/audio_df.csv')